In [1]:
from __future__ import print_function

Load in the Python modules necessary to load the data (gzip and yaml), process the data (numpy and pandas), and plot the data (toyplot).

In [2]:
import gzip
import yaml
import numpy
import pandas
import toyplot.pdf

print("yaml version:    ", yaml.__version__)
print("numpy version:   ", numpy.__version__)
print("pandas version:  ", pandas.__version__)
print("toyplot version: ", toyplot.__version__)

yaml version:     3.12
numpy version:    1.14.0
pandas version:   0.22.0
toyplot version:  0.16.0


## Ingest Data

Load the data, which is output into a YAML file. (Actually, there are several runs that have been concatinated into a YAML file that has a sequence in its top level.)

In [3]:
filename = 'miniGraphics-skybridge-vn-no-compress.yaml.gz'
yaml_data = yaml.load(gzip.open(filename))
data = pandas.DataFrame(yaml_data)

The YAML data is hierarchical. The basic yaml reader to DataFrame just embeds dictionaries and lists in DataFrame columns. Fix that by expanding the data of these columns into new columns.

In [4]:
def expand_single_column(original_data, column_to_expand):

    expanded_data = pandas.DataFrame()
    for index in original_data.index:
        sub_table = pandas.DataFrame(original_data[column_to_expand][index])
        for column in original_data.columns:
            if column != column_to_expand:
                sub_table[column] = numpy.full(sub_table.index.shape,
                                               original_data[column][index],
                                               dtype=original_data[column].dtype)
        expanded_data = expanded_data.append(sub_table, ignore_index=True)
    return expanded_data

def flatten_table(original_data):
    flat_data = original_data
    for column_name in original_data.columns:
        if isinstance(flat_data[column_name][0], list):
            flat_data = expand_single_column(flat_data, column_name)
    return flat_data

In [5]:
data = flatten_table(data)

Add a column that gives a human-readable name to each image resolution.

In [6]:
image_height_names = {
    500: 'Desktop Window',
    1080: 'HDTV',
    4320: '8K UHD',
}

data['image-size'] = data['image-height'].map(image_height_names)

Rename the algorithms from the identifiers the program wrote out to the strings used in the paper. Note that there are some extras in the data that we are ignoring.

In [7]:
algorithm_names = {
    '2-3-SwapBase': '2-3 Swap',
    'BinarySwapFold': 'Naive',
    'BinarySwapTelescoping': 'Telescoping',
    'BinarySwapRemainder': 'Remainder',
    'IceTBase': 'IceT'
}

data['composite-algorithm'] = data['composite-algorithm'].map(algorithm_names)

Print a summary of the table data. There are multiple ways that Jupyter and pandas will report a summary of a table, but I find this method the most effective. It prints out every column. Then for all columns with a "small" number of unique values, it gives those values. This latter information really helps identify the proper way to group values.

In [8]:
import IPython.display

data_description = ''

for column_name in data.columns:
    data_description = data_description + '**' + column_name + '** '
    unique_values = data[column_name].unique()
    if (len(unique_values) < 10):
        for value in unique_values:
            data_description = data_description + str(value) + ' '
    data_description = data_description + ' \n'
    
IPython.display.display(IPython.display.Markdown(data_description))

**color-buffer-format** byte  
**composite-algorithm** Naive Remainder 2-3 Swap  
**composite-seconds**  
**construct-tree-seconds**  
**depth-buffer-format** float  
**gather-seconds**  
**geometry** box  
**geometry-distribution** duplicate  
**geometry-overlap** -0.05  
**image-compression** False  
**image-height** 500 1080 4320  
**image-width** 500 1920 7680  
**num-processes**  
**num-triangles**  
**paint-seconds**  
**painter** simple  
**partial-composite-seconds**  
**phi-rotation**  
**random-seed** 17627  
**rendering-order-dependent** False  
**start-time**  
**theta-rotation**  
**total-seconds**  
**trial-num**  
**zoom** 1.5  
**image-size** Desktop Window HDTV 8K UHD  


## Plot Data

We are plotting the time it takes to do a "partial composite" (that is the time to blend all the pixels, but the pixels are left distributed across all the processes).

The first thing we want to do is to average the time it took over all trials. This is easily done with a pivot table.

In [9]:
average_partial_composite = data.pivot_table(
    values='partial-composite-seconds',
    index='num-processes',
    columns=[
        'image-size',
        'composite-algorithm',
    ],
    aggfunc='mean',
)

print(average_partial_composite.index)
print(average_partial_composite.columns)

Int64Index([1152, 1296, 1536, 1728, 2048, 2304, 2592, 3072, 3456, 3888, 4096,
            4608, 5184, 6144, 6912, 7776, 8192],
           dtype='int64', name=u'num-processes')
MultiIndex(levels=[[u'8K UHD', u'Desktop Window', u'HDTV'], [u'2-3 Swap', u'Naive', u'Remainder']],
           labels=[[0, 0, 0, 1, 1, 1, 2, 2, 2], [0, 1, 2, 0, 1, 2, 0, 1, 2]],
           names=[u'image-size', u'composite-algorithm'])


Make a grouping structure of the data so we can pull out the actual data values for each trial.

In [10]:
grouped_data = data.groupby(['image-size', 'composite-algorithm'])

Plot data.

In [11]:
image_size = 'HDTV'
algorithms = ['2-3 Swap', 'Naive', 'Remainder']

canvas = toyplot.Canvas('4.5in', '3in')

axes = canvas.cartesian(
    xlabel='Number of Processes',
    ylabel='Partial Composite Time (seconds)',
    xscale='log',
    bounds=(45,-15,15,-50),
)

#axes.x.ticks.locator = toyplot.locator.Log(base=2, format='{:.0f}')
axes.x.ticks.locator = toyplot.locator.Explicit(
    locations=[1024, 2048, 4096, 8192],
)
axes.y.domain.min = 0

for algorithm in algorithms:
    # Plot line following average
    data_series = average_partial_composite[image_size, algorithm]
    x = data_series.index
    y = numpy.array(data_series)
    axes.plot(x, y)
    
    # Some adjustments for correct label placement
    if algorithm == 'Naive':
        vert_align = 'top'
    else:
        vert_align = 'middle'
    
    # Put label at end of series
    axes.text(
        x[-1],
        y[-1],
        algorithm + '&#160;',
        style={
            'text-anchor':'start',
            '-toyplot-anchor-shift':'5px',
            '-toyplot-vertical-align': vert_align,
        },
    )
    
    # Put a dot at every recorded sample.
    scatter_data = data.loc[grouped_data.groups[(image_size, algorithm)]]
    x = scatter_data['num-processes']
    y = scatter_data['partial-composite-seconds']
    axes.scatterplot(x, y, opacity=1)

<path d="M 63.00996881782484 155.06814198 L 81.01993763564982 158.8788195 L 106.99886013337164 163.21849315999998 L 125.00882895119662 161.69278068 L 150.9877514489186 167.76475402 L 168.99772026674344 147.56959750000001 L 187.00768908456843 143.8934871 L 212.98661158229024 152.17486416 L 230.99658040011525 152.70605016000002 L 249.0065492179402 145.88547920000002 L 256.97550289783703 140.1646372 L 274.9854717156621 131.7393404 L 292.995440533487 128.0820958 L 318.97436303120884 107.16558780000003 L 336.9843318490339 100.82467200000002 L 354.99430066685863 79.76874439999997 L 362.9632543467556 37.50284080000003" style="stroke:rgb(40%,76.1%,64.7%);stroke-opacity:1.0;stroke-width:2.0"> 2-3 Swap  <path d="M 63.00996881782484 147.43295202 L 81.01993763564982 146.1999092 L 106.99886013337164 146.2380868 L 125.00882895119662 144.7529514 L 150.9877514489186 180.18866828 L 168.99772026674344 145.7289778 L 187.00768908456843 137.7723824 L 212.98661158229024 137.24998259999995 L 230.99658040011525 137.029168 L 249.0065492179402 141.28084139999999 L 256.97550289783703 171.29490646 L 274.9854717156621 143.75881739999997 L 292.995440533487 128.6655084 L 318.97436303120884 119.90187600000002 L 336.9843318490339 127.873457 L 354.99430066685863 125.8264954 L 362.9632543467556 165.08427172" style="stroke:rgb(98.8%,55.3%,38.4%);stroke-opacity:1.0;stroke-width:2.0"> Naive  <path d="M 63.00996881782484 181.22550678000002 L 81.01993763564982 180.54866164 L 106.99886013337164 180.79723974 L 125.00882895119662 179.28015290000002 L 150.9877514489186 177.00568224 L 168.99772026674344 177.41324596 L 187.00768908456843 168.93952248 L 212.98661158229024 174.26776852 L 230.99658040011525 173.59184213999998 L 249.0065492179402 174.2377215 L 256.97550289783703 170.6355579 L 274.9854717156621 175.8351151 L 292.995440533487 166.80878424 L 318.97436303120884 164.99743552 L 336.9843318490339 166.86345046 L 354.99430066685863 169.08160024 L 362.9632543467556 156.96175986" style="stroke:rgb(55.3%,62.7%,79.6%);stroke-opacity:1.0;stroke-width:2.0"> Remainder  <

In [12]:
toyplot.pdf.render(canvas, 'no-compress-hdtv.pdf')

In [13]:
image_size = '8K UHD'
algorithms = ['2-3 Swap', 'Naive', 'Remainder']

canvas = toyplot.Canvas('4.5in', '3in')

axes = canvas.cartesian(
    xlabel='Number of Processes',
    ylabel='Partial Composite Time (seconds)',
    xscale='log',
    bounds=(45,-15,15,-50),
)

#axes.x.ticks.locator = toyplot.locator.Log(base=2, format='{:.0f}')
axes.x.ticks.locator = toyplot.locator.Explicit(
    locations=[1024, 2048, 4096, 8192],
)
axes.y.domain.min = 0

for algorithm in algorithms:
    # Plot line following average
    data_series = average_partial_composite[image_size, algorithm]
    x = data_series.index
    y = numpy.array(data_series)
    axes.plot(x, y)
    
    if algorithm == 'Remainder':
        vert_shift = '40%'
    elif algorithm == 'Naive':
        vert_shift = '-10%'
    else:
        vert_shift = '0'
    
    # Put label at end of series
    axes.text(
        x[-1],
        y[-1],
        algorithm + '&#160;',
        style={
            'text-anchor':'start',
            '-toyplot-anchor-shift':'5px',
            'baseline-shift': vert_shift,
        },
    )
    
    # Put a dot at every recorded sample.
    scatter_data = data.loc[grouped_data.groups[(image_size, algorithm)]]
    x = scatter_data['num-processes']
    y = scatter_data['partial-composite-seconds']
    axes.scatterplot(x, y, opacity=1)

<path d="M 63.00996881782484 158.15667116666668 L 81.01993763564982 157.28903391666668 L 106.99886013337164 153.08055933333335 L 125.00882895119662 138.5204619166667 L 150.9877514489186 149.62868658333332 L 168.99772026674344 150.7357515 L 187.00768908456843 130.70670916666666 L 212.98661158229024 113.82389950000002 L 230.99658040011525 125.59501024999999 L 249.0065492179402 118.5801921666667 L 256.97550289783703 141.30606641666665 L 274.9854717156621 143.9047969166667 L 292.995440533487 84.18617741666668 L 318.97436303120884 120.93107675 L 336.9843318490339 128.89316166666669 L 354.99430066685863 136.83448900000002 L 362.9632543467556 143.97619408333333" style="stroke:rgb(40%,76.1%,64.7%);stroke-opacity:1.0;stroke-width:2.0"> 2-3 Swap  <path d="M 63.00996881782484 173.49492708333335 L 81.01993763564982 173.41760183333335 L 106.99886013337164 173.38649333333336 L 125.00882895119662 150.19703925 L 150.9877514489186 186.59959641666666 L 168.99772026674344 172.87009966666665 L 187.00768908456843 146.14375408333333 L 212.98661158229024 142.05432433333334 L 230.99658040011525 143.55227108333332 L 249.0065492179402 139.74491774999998 L 256.97550289783703 179.50940433333332 L 274.9854717156621 172.0228855 L 292.995440533487 127.52578141666665 L 318.97436303120884 120.05414783333333 L 336.9843318490339 134.54792133333336 L 354.99430066685863 122.91798816666666 L 362.9632543467556 155.63035991666666" style="stroke:rgb(98.8%,55.3%,38.4%);stroke-opacity:1.0;stroke-width:2.0"> Naive  <path d="M 63.00996881782484 176.5993100833333 L 81.01993763564982 178.6359505 L 106.99886013337164 175.51850341666665 L 125.00882895119662 171.93190149999998 L 150.9877514489186 172.07895141666668 L 168.99772026674344 173.03543558333334 L 187.00768908456843 169.30748883333334 L 212.98661158229024 155.77947258333333 L 230.99658040011525 160.59489741666667 L 249.0065492179402 161.28651333333335 L 256.97550289783703 160.399345 L 274.9854717156621 160.13172641666665 L 292.995440533487 136.8508980833333 L 318.97436303120884 135.7757593333333 L 336.9843318490339 159.6640025 L 354.99430066685863 129.27393416666663 L 362.9632543467556 135.78131575" style="stroke:rgb(55.3%,62.7%,79.6%);stroke-opacity:1.0;stroke-width:2.0"> Remainder  <g class="toyplot-Dat

In [14]:
toyplot.pdf.render(canvas, 'no-compress-8k.pdf')